# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて、言語モデルを製作し、一般的な言葉当てゲーム「20の質問」をプレイするためのロジックを実装しています。特に、モデルの状態の管理、質問と回答の生成、及び推測を行うエージェントを作成することに取り組んでいます。

### 問題へのアプローチ
- **依存関係のインストール**: Kaggle環境に必要なライブラリ（`rigging`、`transformers`、`Kaggle`など）をインストール。
- **シークレット情報の取得**: Hugging Face及びKaggleのAPIトークンを取得します。
- **モデルのダウンロード**: Hugging Face Hubから特定のモデル（`Meta-Llama-3-8B-Instruct-bnb-8bit`）をダウンロードしてローカルに保存します。
- **モデルの読み込みと検証**: ダウンロードしたモデルを用いて簡単なチャットを実行し、正常に動作するか確認します。

### 使用しているライブラリ
- **Rigging**: モデルの質問生成や応答の管理を行うライブラリ。
- **Hugging Face**: 事前トレーニングされたモデルをダウンロードし、使用するためのライブラリ。
- **pydantic**: データの検証や設定の管理を行うためのライブラリ。

### 実装の詳細
- **クラス定義**: `Observation`や`Question`、`Answer`、`Guess`といったクラスを実装し、それぞれのデータ構造やバリデーションルールを定義しています。
- **関数の実装**: `ask`、`answer`、`guess`といった関数を定義し、それぞれエージェントが質問を生成したり、回答したり、推測を行うロジックを組んでいます。
- **メインエントリーポイント**: `agent_fn`関数にて、異なるターンの処理を行い、実際のゲームプレイに必要な動作を管理します。

### 出力の整理
最終的に、`submission.tar.gz`が生成され、Kaggleコンペティションへの提出が行われます。このスクリプトは、ゲームロジックを忠実に再現し、言語モデルが効果的に機能するための重要な構成要素を構築しています。

---


# 用語概説 
以下は、Jupyter Notebook内の機械学習・深層学習の専門用語について、初心者がつまずきそうなポイントを解説したものです。

1. **Hugging Faceのトークン (HF_TOKEN)**:
   Hugging Faceが提供するモデルやデータセットにアクセスするための認証情報です。トークンを利用することで、特定のリソースに対する読み取りや書き込みが可能になります。

2. **snapshot_download**:
   Hugging Face Hubからモデルやデータをダウンロードする関数です。このライブラリでは、特定のリポジトリからモデルを簡単に取得できます。

3. **Rigging**:
   特定のフレームワークやライブラリ名で、機械学習モデルの操作を簡素化するために使用されます。このコンテキストでは、質問応答などの処理を行うためのインターフェースを提供しています。

4. **PendingChat**:
   これは、現在のチャットセッションの待機状態を表します。状態管理が含まれており、特定のコンテキストでの質問や回答を追跡します。

5. **BaseModel**:
   Pydanticライブラリが提供するクラスで、データモデルを作成する際に使用されます。データのバリデーションや型チェックを簡略化するための機能を持っています。

6. **field_validator**:
   Pydanticの機能で、特定のフィールドの値を検証するためのメソッドです。この仕組みを利用することで、カスタムバリデーションロジックを簡単に追加できます。

7. **Literal**:
   型ヒントにおいて、特定のリテラル値のリストを指定するためのものです。これにより、一部の特定の文字列や値のみが許可されることを示します。

8. **str_strip**:
   標準の文字列型にホワイトスペースを除去する制約を追加したカスタム型です。データの整合性を保つために使われることが多いです。

9. **XML形式**:
   データの構造を定義するためのマークアップ言語です。ここでは履歴データをXML形式で出力することによって、データの可読性を高め、他のシステムとインターフェースしやすくします。

10. **ファイナリゼーション**:
   クラスのメソッドの実装において、エラー処理やリカバリのオプションを用意する手法です。コードのロバスト性を高め、予期しない状況に対応するために重要です。

11. **二分割探索**:
   情報を効率的に得るために使用される戦略です。このプロセスは、質問を通じて探索空間を段階的に狭めていくことを意味し、情報収集を効果的に行う手法です。

これらの用語は、特にこのノートブックの特定の文脈に関連しているため、初心者は理解しておくと役立つでしょう。

---


In [ ]:
# 依存関係のインストール

!pip install -U \
    -t /kaggle/tmp/lib \
    rigging \
    kaggle \
    transformers \
    accelerate \
    -i https://pypi.org/simple/ bitsandbytes

In [ ]:
# シークレット情報の取得

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN = secrets.get_secret("HF_TOKEN")

KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")

In [ ]:
# モデルのダウンロード

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)  # 既存のモデルパスがあれば削除
g_model_path.mkdir(parents=True)  # 新しいモデルパスを作成

snapshot_download(
    repo_id="alokabhishek/Meta-Llama-3-8B-Instruct-bnb-8bit",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=HF_TOKEN  # Hugging Faceのトークンを使用
)

In [ ]:
# モデルの読み込み検証

import sys

sys.path.insert(0, "/kaggle/tmp/lib")  # ライブラリパスを追加

import rigging as rg  # Riggingライブラリのインポート

generator = rg.get_generator("transformers!/kaggle/tmp/model,device_map=cuda:0")  # ジェネレーターの取得
chat = generator.chat("Say Hello!").run()  # チャットを実行

print(chat.last)  # 最新のチャットメッセージを表示

In [ ]:
%%writefile main.py

import itertools
import os
import sys
import typing as t
from pathlib import Path

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")
g_model_path = g_temp_path / "model"

if g_agent_path.exists():
    sys.path.insert(0, str(g_agent_path / "lib"))  # エージェントのライブラリパスを追加
    g_model_path = g_agent_path / "model"
else:
    sys.path.insert(0, str(g_temp_path / "lib"))  # 一時的なライブラリパスを追加

import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_generator_id = f"transformers!{g_model_path},trust_remote_code=True,max_tokens=1024,temperature=1.0,top_k=256"

# 型

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int  # 現在のステップ
    role: t.Literal["guesser", "answerer"]  # 役割（推測者または回答者）
    turnType: t.Literal["ask", "answer", "guess"]  # ターンの種類
    keyword: str  # キーワード
    category: str  # カテゴリー
    questions: list[str]  # 質問のリスト
    answers: list[str]  # 回答のリスト
    guesses: list[str]  # 推測のリスト
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])  # すべてのリストが空か確認
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")  # 履歴を取得

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "none yet."  # XML形式で履歴を取得


class Answer(rg.Model):
    content: t.Literal["yes", "no", "maybe"]  # 回答の内容

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:  # 回答の内容を検証
        for valid in ["yes", "no", "maybe"]:  # 有効な回答を定義
            if v.lower().startswith(valid):
                return valid
        raise ValueError("Invalid answer, must be one of 'yes', 'no', 'maybe'")  # 無効な回答の場合エラーを投げる

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no/maybe**{Answer.xml_end_tag()}"  # XML例を返す


class Question(rg.Model):
    content: str_strip  # 質問の内容

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # 質問のXML例


class Guess(rg.Model):
    content: str_strip  # 推測の内容

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="**thing/place/person**").to_pretty_xml()  # 推測のXML例


# 関数


def ask(base: rg.PendingChat, observation: Observation) -> str:
    chat = (
        base.fork(
            f"""\
            あなたは現在、次の質問をしています。

            <game-history>
            {observation.get_history_as_xml(skip_guesses=True)}
            </game-history>

            上記の履歴に基づいて、次に最も有用なはい/いいえの質問を尋ね、以下の形式に置いてください:
            {Question.xml_example()}

            - あなたの応答は、最も情報を得るための集中した質問である必要があります
            - 質問では一般的な内容から始めてください
            - 残りの探索空間を常に二分割するようにしてください
            - 前の質問と回答に注意を払ってください

            次の質問は何ですか？
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Question).content


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no", "maybe"]:
    last_question = observation.questions[-1]  # 最後の質問を取得
    chat = (
        base.fork(
            f"""\
            このゲームの秘密の言葉は "{observation.keyword}" です。

            あなたは現在、上記の作業についての質問に回答しています。

            次の質問は "{last_question}" です。

            上記のはい/いいえの質問に回答し、以下の形式に置いてください:
            {Answer.xml_example()}

            - あなたの応答は、上記のキーワードに基づいて正確である必要があります
            - 常に"はい"、"いいえ"、または"多分"で回答してください

            答えは何ですか？
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Answer).content


def guess(base: rg.PendingChat, observation: Observation) -> str:
    pending = (
        base.fork(
            f"""\
            あなたは現在、キーワードのインフォームド・ゲスを行っています。

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            上記の履歴に基づいて、キーワードの次の最適な推測を生成し、以下の形式に置いてください:
            {Guess.xml_example()}

            - 上記の履歴に基づいて繰り返しの推測を避けてください
            - 推測は具体的な人、場所、または物であるべきです

            あなたの推測は何ですか？
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)
        .run()
    )
        
    return chat.last.parse(Guess).content

# ジェネレーター

generator = rg.get_generator(g_generator_id)

# エントリーポイント

def agent_fn(obs: t.Any, _: t.Any) -> str:
    observation = Observation(**obs.__dict__)  # 観察データを生成
    
    try:
        base = generator.chat("""\
            あなたは20の質問ゲームの才能あるプレイヤーです。あなたは正確で、集中力があり、構造的なアプローチを持っています。あなたは有用な質問を作成し、推測を行い、キーワードに関する質問に回答します。
            
            """
        )
    
        match observation.turnType:  # ターンの種類によって処理を分ける
            case "ask":
                return ask(base, observation)  # 質問をする場合
            case "answer":
                if not observation.keyword:  # キーワードがない場合
                    return "maybe"  # 多分を返す
                return answer(base, observation)  # 回答をする場合
            case "guess":
                return guess(base, observation)  # 推測をする場合
            case _:
                raise ValueError("Unknown turn type")  # 不明なターンタイプの場合エラーを投げる
    except Exception as e:
        print(str(e), file=sys.stderr)  # エラーを標準エラーに出力
        raise

In [ ]:
!apt install pigz pv  # pigzとpvをインストール

In [ ]:
# submission.tar.gzを作成する

!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib \
    -C /kaggle/working main.py

In [ ]:
# Kaggleに直接プッシュ

!KAGGLE_USERNAME={KAGGLE_USERNAME} \
 KAGGLE_KEY={KAGGLE_KEY} \
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"

---

# コメント 

> ## Marília Prata
> 
> Rigging with Llama3を紹介してくれてありがとう Nick (Nicklanders)。
> 
> 

---